*This notebook is part of  course materials for CS 345: Machine Learning Foundations and Practice at Colorado State University.
Original versions were created by Asa Ben-Hur and updated by Ross Beveridge.
The content is availabe [on GitHub](https://github.com/asabenhur/CS345).*

*The text is released under the [CC BY-SA license](https://creativecommons.org/licenses/by-sa/4.0/), and code is released under the [MIT license](https://opensource.org/licenses/MIT).*

<a href="https://colab.research.google.com/github//asabenhur/CS345/blob/master/fall24/notebooks/module01_06_perceptron.ipynb">
  <img align="left" src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/>
</a>

# Predicting Valorant Match Outcomes Using Performance Metrics

By Hallie Gurr and Rose Ordway

In [54]:
from json import encoder
import sqlite3
import pandas as pd
from sklearn.preprocessing import OneHotEncoder
import ast
import numpy as np

#Helper functions
def removeNanRows(X):
    df = pd.DataFrame(X)
    df = df.dropna()
    return df.values

In [70]:
#Database connection
db_path = "data/valorant.sqlite"
conn = sqlite3.connect(db_path)

# Check existing tables names
tables = pd.read_sql_query(
    "SELECT name FROM sqlite_master WHERE type='table';", conn
)
print(tables)

              name
0          Matches
1            Games
2      Game_Rounds
3  Game_Scoreboard


In [ ]:
#Get Scoreboard Data
def getScoreboard(conn, limit=None):
    #Looking at GameID, PlayerID, Agent, Kills, Deaths, Assists, HS_Percent, Econ
    headers = ["GameID", "PlayerID", "Agent", "Kills", "Deaths", 
           "Assists", "HS_Percent", "Econ"]

    selections = ", ".join(headers)

    conditions = " AND ".join([f"{h} IS NOT NULL" for h in headers])

    query = f"""
    SELECT {selections}
    FROM Game_Scoreboard
    WHERE {conditions}
    LIMIT {-1 if limit is None else limit};
    """
    scoreboard = pd.read_sql_query(query, conn)
    return scoreboard

def getAgents(conn, limit=None):
    scoreboard = getScoreboard(conn, limit=limit)
    agents = scoreboard["Agent"]
    encoder = OneHotEncoder(sparse_output=False)
    agent_encoded = encoder.fit_transform(agents.values.reshape(-1, 1))
    return agent_encoded

def agentNames(conn, limit=None):
    scoreboard = getScoreboard(conn, limit=limit)
    agents = scoreboard["Agent"]
    encoder = OneHotEncoder(sparse_output=False)
    encoder.fit(agents.values.reshape(-1, 1))
    return encoder.categories_[0]

def MLScoreBoardData(conn, limit=None):
    scoreboard = getScoreboard(conn, limit=limit)

    features = scoreboard[["Kills", "Deaths", "Assists", "HS_Percent", "Econ"]]
    features = features.values

    X = np.hstack((features, getAgents(conn, limit=limit)))
    X = removeNanRows(X)
    return X

mlscoreboard = MLScoreBoardData(conn, limit=10000)
df = pd.DataFrame(mlscoreboard, columns=[
    "Kills", "Deaths", "Assists", "HS_Percent", "Econ"
] + list(agentNames(conn, limit=10000)))
print(df)



      Kills  Deaths  Assists  HS_Percent  Econ  astra  breach  brimstone  \
0      24.0    10.0      3.0        0.31  74.0    0.0     0.0        0.0   
1      16.0    10.0      7.0        0.16  67.0    0.0     0.0        0.0   
2      17.0     9.0      8.0        0.27  58.0    0.0     0.0        0.0   
3      17.0    12.0      2.0        0.19  48.0    0.0     0.0        0.0   
4       5.0    13.0      3.0        0.22  21.0    0.0     0.0        0.0   
...     ...     ...      ...         ...   ...    ...     ...        ...   
9995   28.0    27.0      6.0        0.23  50.0    0.0     0.0        0.0   
9996   27.0    22.0      5.0        0.21  48.0    0.0     0.0        0.0   
9997   17.0    24.0     10.0        0.19  31.0    0.0     0.0        0.0   
9998   15.0    24.0      6.0        0.41  30.0    0.0     0.0        0.0   
9999   30.0    24.0      9.0        0.20  57.0    0.0     0.0        0.0   

      chamber  cypher  ...  killjoy  omen  phoenix  raze  reyna  sage  skye  \
0       